Guidance on using this dataset: https://www.kaggle.com/code/acelevin/identifying-playing-cards

Download dataset from: https://www.kaggle.com/datasets/gunhcolab/object-detection-dataset-standard-52card-deck/data

In [5]:
import tensorflow as tf
import pickle
from PIL import Image
import os
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import cv2

In [6]:
#hyper parameters:
BATCH_SIZE = 32
NUM_EPOCHS = 20

### Model Architecture

In [7]:
class CardPredictor(tf.keras.Model):
    def __init__(self):
        super(CardPredictor, self).__init__()
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
        
        data_augmentation = tf.keras.models.Sequential([
                                tf.keras.layers.RandomRotation(0.1),
                                tf.keras.layers.RandomZoom(0.1),
                            ])
        
        self.architecture = [        
                tf.keras.layers.InputLayer((300, 300, 3)),
                data_augmentation,
                
                tf.keras.layers.Conv2D(32, 3, padding='same', use_bias=False),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.Conv2D(32, 3, padding='same', use_bias=False),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                
                tf.keras.layers.MaxPooling2D(2, 2),
                tf.keras.layers.Dropout(0.2),
                
                tf.keras.layers.Conv2D(64, 3, padding='same', use_bias=False),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.Conv2D(64, 3, padding='same', use_bias=False),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                
                tf.keras.layers.MaxPooling2D(2, 2),
                tf.keras.layers.Dropout(0.3),
                
                tf.keras.layers.Conv2D(128, 3, padding='same', use_bias=False),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.Conv2D(128, 3, padding='same', use_bias=False),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                
                tf.keras.layers.MaxPooling2D(2, 2),
                tf.keras.layers.Dropout(0.4),
                
                tf.keras.layers.GlobalAveragePooling2D(),
                
                tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
                tf.keras.layers.Dropout(0.5),
                tf.keras.layers.Dense(52, activation='softmax')        
                ]
        
        self.sequential = tf.keras.Sequential(self.architecture, name="card_predictor")
        
    def call(self, x):
        """ Passes input image through the network. """
        return self.sequential(x)

    @staticmethod
    def loss_fn(labels, predictions): 
           """ Loss function for the model. """
           return tf.keras.losses.sparse_categorical_crossentropy(labels, predictions)

### Dataset Loading

In [8]:
with open('train.pkl', 'rb') as file:
    data = pickle.load(file)
    
new_data = {}
for key, inner_dict in data.items():
    img_path = inner_dict['img_path']
    value = inner_dict['class_label']
    new_data[img_path] = value

In [9]:
dataset = tf.data.Dataset.from_tensor_slices((new_data.keys(), new_data.values()))

def load_train_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [300, 300])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

dataset = dataset.map(load_train_image, num_parallel_calls=tf.data.AUTOTUNE).shuffle(buffer_size=10000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

2025-04-29 20:59:27.620325: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-04-29 20:59:27.620351: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-29 20:59:27.620357: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-29 20:59:27.620377: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-29 20:59:27.620399: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
# this cell it optional
# this checks that the dataset is balanced
import collections
v_counts = collections.Counter()
t_counts = collections.Counter()

for _, label in val_dataset:
    label = label.numpy().tolist() 
    v_counts.update(label)

for _, label in train_dataset:
    label = label.numpy().tolist() 
    t_counts.update(label)
    
print("Label counts in validation dataset:")
for i in range(52):
    print(f"Label {i}: {v_counts[i]}, {t_counts[i]}")

2025-04-29 20:59:32.880452: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: imgs/single/6.png; No such file or directory
2025-04-29 20:59:32.880471: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: imgs/single/4.png; No such file or directory
2025-04-29 20:59:32.880482: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: imgs/single/3.png; No such file or directory
2025-04-29 20:59:32.880492: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: imgs/single/1.png; No such file or directory
2025-04-29 20:59:32.880500: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: imgs/single/5.png; No such file or directory
2025-04-29 20:59:32.880512: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES fai

NotFoundError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Error in user-defined function passed to ParallelMapDatasetV2:1 transformation with iterator: Iterator::Root::Prefetch::FiniteSkip::Prefetch::BatchV2::Shuffle::ParallelMapV2: imgs/single/0.png; No such file or directory
	 [[{{node ReadFile}}]] [Op:IteratorGetNext] name: 

### Model Loading and Training

In [11]:
model = CardPredictor()
#model = tf.keras.models.load_model("MODEL_PATH")

model.compile(optimizer=model.optimizer, loss=model.loss_fn, metrics=['accuracy'])

In [12]:
model.fit(train_dataset, 
          validation_data=val_dataset, 
          epochs=NUM_EPOCHS, 
          verbose=1)

Epoch 1/20


2025-04-29 20:59:39.694162: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-04-29 20:59:40.273628: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: imgs/single/3.png; No such file or directory
2025-04-29 20:59:40.273642: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: imgs/single/2.png; No such file or directory
2025-04-29 20:59:40.273648: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: imgs/single/1.png; No such file or directory
2025-04-29 20:59:40.273691: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: imgs/single/6.png; No such file or directory
2025-04-29 20:59:40.273710: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at whole_file_read_ops.

NotFoundError: Graph execution error:

Detected at node ReadFile defined at (most recent call last):
<stack traces unavailable>
Detected at node ReadFile defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) NOT_FOUND:  Error in user-defined function passed to ParallelMapDatasetV2:1 transformation with iterator: Iterator::Root::Prefetch::FiniteTake::Prefetch::BatchV2::Shuffle::ParallelMapV2: imgs/single/0.png; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]]
	 [[StatefulPartitionedCall/adam/Add_28/_84]]
  (1) NOT_FOUND:  Error in user-defined function passed to ParallelMapDatasetV2:1 transformation with iterator: Iterator::Root::Prefetch::FiniteTake::Prefetch::BatchV2::Shuffle::ParallelMapV2: imgs/single/0.png; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]]
0 successful operations.
0 derived errors ignored. [Op:__inference_multi_step_on_iterator_18878]

In [13]:
model.save_weights('model_weights.weights.h5')

## Video capture and classification

In [14]:
model = CardPredictor()
model.build((None, 300, 300, 3))
model.load_weights('model_weights.weights.h5')

/Users/jad/miniforge3/envs/csci1430/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 46 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [16]:
class CardPredictor(tf.keras.Model):
    def __init__(self):
        super(CardPredictor, self).__init__()
        self.architecture = [        
            tf.keras.layers.InputLayer((300, 300, 3)),
            tf.keras.layers.Conv2D(32, 3, padding='same', use_bias=False),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(32, 3, padding='same', use_bias=False),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Conv2D(64, 3, padding='same', use_bias=False),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3, padding='same', use_bias=False),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Conv2D(128, 3, padding='same', use_bias=False),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(128, 3, padding='same', use_bias=False),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Dropout(0.4),
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(52, activation='softmax')        
        ]
        self.sequential = tf.keras.Sequential(self.architecture)

    def call(self, x):
        return self.sequential(x)

# Load model
model = CardPredictor()
model.build((None, 300, 300, 3))
model.load_weights('model_weights.weights.h5')

class_names = [f'{v}{s}' for v in '23456789TJQKA' for s in 'CDHS'] # this returns what the card is from the number returned by the model, might be wrong, check labels to make sure it is correct, or use the dictionary instead.  

# === Start camera ===
cam = cv2.VideoCapture(2)

if not cam.isOpened():
    print("Error: Could not open camera.")
    exit()

print("Press 'c' to capture and classify a card.")
print("Press 'q' to quit.")

while True:
    ret, frame = cam.read()
    if not ret:
        print("Error: Failed to read frame.")
        break

    # Show the camera feed
    cv2.imshow("Live Feed - Press 'c' to classify, 'q' to quit", frame)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        print("Exiting.")
        break
    elif key == ord('c'):
        print("Capturing frame...")

        # Preprocess the frame
        img = cv2.resize(frame, (300, 300))
        img = img.astype('float32') / 255.0
        img = np.expand_dims(img, axis=0)

        # Predict
        pred = model(img, training=False)
        predicted_class = tf.argmax(pred, axis=1).numpy()[0]
        # check dataset to fix the indexing here.
        class_name = class_names[predicted_class]

        # Overlay prediction and display result on the top right corner of the image. 
        display_frame = frame.copy()
        cv2.putText(display_frame, f"Prediction: {class_name}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow("Prediction", display_frame)
        cv2.waitKey(1500)  # Show prediction for 1.5 seconds

cam.release()
cv2.destroyAllWindows()

Press 'c' to capture and classify a card.
Press 'q' to quit.
Capturing frame...
Capturing frame...
Capturing frame...
Exiting.


: 